In [1]:
# import
import csv
import math
import random
import sys
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

from sklearn.neighbors import NearestNeighbors

import torch
import torch.nn as nn
import torch.nn.functional as F

from IPython.display import clear_output

%matplotlib inline

random.seed

<bound method Random.seed of <random.Random object at 0x10204a018>>

In [2]:
def IoW(gt, coor, radius_gt, radius):
    ### IoW cross section calculation
#     print("s-1")
    cross_list_0 = [(gt[0]-radius_gt, 0), (gt[0]+radius_gt, 0)]
    cross_list_1 = [(gt[1]-radius_gt, 0), (gt[1]+radius_gt, 0)]
#     print("s-2")
    cross_list_0.append((coor[0]-radius, 1))
    cross_list_0.append((coor[0]+radius, 1))
    cross_list_1.append((coor[1]-radius, 1))
    cross_list_1.append((coor[1]+radius, 1))
#     print("s-3")                 
    cross_list_0.sort(key = lambda x : x[0])
    cross_list_1.sort(key = lambda x : x[0])
#     print("s-4") 
    if (cross_list_0[0][1] != cross_list_0[1][1] and cross_list_1[0][1] != cross_list_1[1][1]):
        return (cross_list_0[2][0] - cross_list_0[1][0]) * (cross_list_1[2][0] - cross_list_1[1][0]) / ((radius*2)**2)
    else:
        return 0.
    
    
def distance_progress(coor_gt, coor_cur, coor_next):
    dis_cur = math.sqrt((coor_gt[0]-coor_cur[0])**2 + (coor_gt[1]-coor_cur[1])**2)
    dis_next = math.sqrt((coor_gt[0]-coor_next[0])**2 + (coor_gt[1]-coor_next[1])**2)
    return dis_cur - dis_next


def right_action(coor_gt, coor_cur, radius_gt, radius):
    next_coordinate = coor_cur.copy()
    action = 0
    next_coordinate[0] -= radius/2.
    next_coordinate[1] += radius/2.
    area = IoW(coor_gt, next_coordinate, radius_gt, radius*0.7)

    next_coordinate = coor_cur.copy()
    next_coordinate[0] += radius/2.
    next_coordinate[1] -= radius/2.
    a = IoW(coor_gt, next_coordinate, radius_gt, radius*0.7)
    if area < a:
        action = 1
        area = a

    next_coordinate = coor_cur.copy()
    next_coordinate[0] -= radius/2.
    next_coordinate[1] -= radius/2.
    a = IoW(coor_gt, next_coordinate, radius_gt, radius*0.7)
    if area < a:
        action = 2
        area = a
    
    next_coordinate = coor_cur.copy()
    next_coordinate[0] += radius/2.
    next_coordinate[1] += radius/2.
    a = IoW(coor_gt, next_coordinate, radius_gt, radius*0.7)
    if area < a:
        action = 3
        area = a

    next_coordinate = coor_cur.copy()
    next_coordinate = coordinate
    a = IoW(coor_gt, next_coordinate, radius_gt, radius*.7)
    if area < a:
        action = 4
        area = a
    return action
    
    
    
class Net(nn.Module):
    def __init__(self, n_states, n_actions, n_hidden):
        super(Net, self).__init__()

        # 輸入層 (state) 到隱藏層，隱藏層到輸出層 (action)
        self.fc1 = nn.Linear(n_states, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden)
        self.out = nn.Linear(n_hidden, n_actions)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x) # ReLU # activation
        x = self.fc2(x)
        x = F.relu(x)
        actions_value = self.out(x)
        return actions_value


class DQN(object):
    def __init__(self, n_states, n_actions, n_hidden, batch_size, lr, gamma, target_replace_iter, memory_capacity):
        self.eval_net, self.target_net = Net(n_states, n_actions, n_hidden), Net(n_states, n_actions, n_hidden)

        self.memory = np.zeros((memory_capacity, n_states * 2 + 2)) # 每個 memory 中的 experience 大小為 (state + next state + reward + action)
        self.optimizer = torch.optim.Adam(self.eval_net.parameters(), lr=lr)
        self.loss_func = nn.MSELoss()
        self.memory_counter = 0
        self.learn_step_counter = 0 # 讓 target network 知道什麼時候要更新

        self.n_states = n_states
        self.n_actions = n_actions
        self.n_hidden = n_hidden
        self.batch_size = batch_size
        self.lr = lr
        self.gamma = gamma
        self.target_replace_iter = target_replace_iter
        self.memory_capacity = memory_capacity

    def choose_action(self, state, epsilon):
        x = torch.unsqueeze(torch.FloatTensor(state), 0)

        # epsilon-greedy
        if (np.random.uniform() < epsilon):
            action = np.random.randint(0, self.n_actions)
        else: 
            actions_value = self.eval_net(x) # 以現有 eval net 得出各個 action 的分數
            action = torch.max(actions_value, 1)[1].data.numpy()[0] # 挑選最高分的 action

        return action
    

    def store_transition(self, state, action, reward, next_state):
        # 打包 experience
        transition = np.hstack((state, [action, reward], next_state))

        # 存進 memory；舊 memory 可能會被覆蓋
        index = self.memory_counter % self.memory_capacity
        self.memory[index, :] = transition
        self.memory_counter += 1
        
    def BP(self, state, next_state, reward):
        q_eval = self.eval_net(torch.FloatTensor(state))
        q_next = self.target_net(torch.FloatTensor(next_state)).detach()
        q_target = reward + self.gamma * q_next
        loss = self.loss_func(q_eval, q_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        
    def learn(self):
        # 隨機取樣 batch_size 個 experience
        sample_index = np.random.choice(self.memory_capacity, self.batch_size)
        b_memory = self.memory[sample_index, :]
        b_state = torch.FloatTensor(b_memory[:, :self.n_states])
        b_action = torch.LongTensor(b_memory[:, self.n_states:self.n_states+1].astype(int))
        b_reward = torch.FloatTensor(b_memory[:, self.n_states+1:self.n_states+2])
        b_next_state = torch.FloatTensor(b_memory[:, -self.n_states:])

        # 計算現有 eval net 和 target net 得出 Q value 的落差
        q_eval = self.eval_net(b_state).gather(1, b_action) # 重新計算這些 experience 當下 eval net 所得出的 Q value
        q_next = self.target_net(b_next_state).detach() # detach 才不會訓練到 target net
        q_target = b_reward + self.gamma * q_next.max(1)[0].view(self.batch_size, 1) # 計算這些 experience 當下 target net 所得出的 Q value
        loss = self.loss_func(q_eval, q_target)

        # Backpropagation
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # 每隔一段時間 (target_replace_iter), 更新 target net，即複製 eval net 到 target net
        self.learn_step_counter += 1
        if self.learn_step_counter % self.target_replace_iter == 0:
            self.target_net.load_state_dict(self.eval_net.state_dict())
            

color = ['#F5B4B3', '#CA885B', '#DAE358', '#9DE358', '#58E39D', \
        '#58E3E1', '#58A2E3', '#5867E3', '#9D58E3', '#E158E3', '#E358B0', '#E35869']

In [ ]:
# for floor in range(0,5):
#     for building in range(0,3):
for floor in range(2,3):
    for building in range(0,1):
        
        ## Count the number of data points in building id & floor id
        data_num = 0
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                data_num += 1
        print(data_num)
        ## if there are no data, continue to next floor 
        if (data_num == 0):
            continue
            
        ## Load data points in
        wifi_loc_time = np.zeros(shape = (data_num, 524))
        i=-1
        with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',')
            for row in spamreader:
                if (row[523] == 'BUILDINGID'):
                    continue
                elif (int(row[523]) is not building or int(row[522]) is not floor):
                    continue
                i = i+1
                if (i > data_num):
                    break
                # wifi
                wifi_loc_time[i-1][:520] = np.array(row[:520])
                # location x, y
                wifi_loc_time[i-1][520:522] = np.array(row[520:522])
                # userID
                wifi_loc_time[i-1][522] = np.array(row[526])
                # time stamp
                wifi_loc_time[i-1][-1] = np.array(row[-1])
        
        ## Sort by time stamp
        wifi_loc_time = wifi_loc_time[wifi_loc_time[:,-1].argsort()]
        
        ## Map boundaries
        longitude_list = np.array([max(wifi_loc_time[:, 520]), -1\
                                   , min(wifi_loc_time[:, 520])])
        latitude_list = np.array([max(wifi_loc_time[:, 521]), -1\
                                   , min(wifi_loc_time[:, 521])])
        
        ## KNN initial calculation
        nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(wifi_loc_time[:,:520])
        distances, indices = nbrs.kneighbors(wifi_loc_time[:,:520])

        ## DQN's hyper para
        n_actions = 5
        # state: RSSI (520), coordinate (2), radius (1), history (50)
        n_states = 520 + 2 + 1 + 50 
        n_hidden = 512
        batch_size = 100
        gamma = 0.1 # reward discount factor
        target_replace_iter = 100
        memory_capacity = 2000
        n_episodes = 10000
        lr = 0.7
        eps = 1.
        max_search_steps = 10
        it = 0
        cost_it = 0
        avg_it = 0
        delta = 0.7
        log_step = 500
        
        radius_gt = 0.5
        
        dqn = DQN(n_states, n_actions, n_hidden, batch_size, lr, gamma, target_replace_iter, memory_capacity)

        ## DQN training
        for k in range(n_episodes):
            print("Epoch - ", k, "eps : ", eps)
            avg_it = 0
            for i in range(len(wifi_loc_time)):
                # some important variables used for training
                Rewards = 0
                Goal = False
                alpha = 0.7
                cost_it = 0
                next_coordinate = np.array([0, 0])
                next_radius = 0
                rect1 = []
                
                ## 1. KNN locates initial coordinates and radius
                Hx = 0.
                Hy = 0.
                for m in range(3):
                    Hx += wifi_loc_time[indices[i, m], 520]
                    Hy += wifi_loc_time[indices[i, m], 521]
                Hx /= 3.
                Hy /= 3.
                coordinate = np.array([Hx, Hy])
                radius = 10.
                # initial history, 5n vector
                history = np.zeros(shape=(5*max_search_steps,), dtype=int)
                
                ## 2. Check initial KNN IoW
                while True:
                    IoW_cur = IoW(wifi_loc_time[i, 520:522], coordinate, radius_gt, radius)
                    if (IoW_cur == 0):
                        radius *= 1.5
                    elif (IoW_cur > delta):
                        print("Precise location!")
                        Goal = True
                        break
                    else:
                        break
                if (Goal == True):
                    continue
                # initial state: RSSI (520), coordinate (2), radius (1), history (50)
                state = np.concatenate((wifi_loc_time[i, :520], coordinate.copy(), np.array([radius]), history.copy()), axis=0)
                
                ## Plot gt region
                if k % log_step == 0:
                    fig = plt.figure()
                    ax = fig.add_subplot(111)
                    plt.xlim(longitude_list[0], longitude_list[2])
                    plt.ylim(latitude_list[0], latitude_list[2])
                    rect0 = plt.Rectangle((wifi_loc_time[i,520]-radius_gt, wifi_loc_time[i,521]-radius_gt), 2*radius_gt, 2*radius_gt, alpha=0.9)
                    rect1.append(plt.Rectangle((coordinate[0]-radius, coordinate[1]-radius), 2*radius, 2*radius, alpha = 0.6, color = color[-1]))
                
                ## 3. Searching starts
                for t in range(max_search_steps):
                    right = right_action(wifi_loc_time[i, 520:522], coordinate, radius_gt, radius)
#                     print(t, "round, right action is ", right)
                    it = 0
                    if (radius < 0.5):
#                         print("Radius is small enough and searching ends")
                        break
                    
                    
                    while True:
                        it += 1
                        ### (1) select an action
                        action = dqn.choose_action(state, eps)
#                         print("[", action, "]", it)
                        ### (1) - 1. New Center
                        ### 0 -> "Up Left"
                        ### 1 -> "Up Right"
                        ### 2 -> "Down Left"
                        ### 3 -> "Down Right"
                        ### 4 -> "Center"
                        next_coordinate = coordinate.copy()
                        if (action == 0):
                            next_coordinate[0] -= radius/2.
                            next_coordinate[1] += radius/2.
                        elif (action == 1):
                            next_coordinate[0] += radius/2.
                            next_coordinate[1] -= radius/2.
                        elif (action == 2):
                            next_coordinate[0] -= radius/2.
                            next_coordinate[1] -= radius/2.
                        elif (action == 3):
                            next_coordinate[0] += radius/2.
                            next_coordinate[1] += radius/2.
                        else:
                            next_coordinate = coordinate
                        ### (1) - 2. New radius
                        next_radius = radius * alpha
                        ### (1) - 3. New IoW
                        next_IoW = IoW(wifi_loc_time[i, 520:522], next_coordinate, radius_gt, next_radius)
#                         print("  IoW", IoW_cur, "->", next_IoW)
                        if (next_IoW > delta and action is right):
#                             print("Precise location!")
                            Goal = True
                            cost_it += it
                            break
                        elif (next_IoW > IoW_cur and action is right):
                            # close score
                            reward = distance_progress(wifi_loc_time[i, 520:522], coordinate, next_coordinate)
#                             print("  [C]  ontinue next round of searching, reward", reward, "\n   location", coordinate, "->", next_coordinate, "\n   radius", radius,"->" , next_radius)
                            IoW_cur = next_IoW
                            next_history = history.copy()
                            one_hot = t*5 + action
                            next_history[one_hot] = 1
                            next_state = np.concatenate((wifi_loc_time[i,:520], next_coordinate, np.array([next_radius]), next_history), axis=0)
                            dqn.store_transition(state.copy(), action, reward, next_state.copy())
                            radius = next_radius
                            coordinate = next_coordinate
                            state = next_state
                            Rewards += reward
                            cost_it += it
                            if k % log_step == 0:
                                # Plot
                                rect1.append(plt.Rectangle((coordinate[0]-radius, coordinate[1]-radius), 2*radius, 2*radius, alpha = 0.6, color = color[t]))
                            break
                        else:
                            reward = distance_progress(wifi_loc_time[i, 520:522], coordinate, next_coordinate)
                            next_history = history.copy()
                            one_hot = t*5 + action
                            next_history[one_hot] = 1
                            next_state = np.concatenate((wifi_loc_time[i,:520], next_coordinate.copy(), np.array([next_radius]), next_history.copy()), axis=0)
                            Rewards += reward
                            # back_propagation
                            dqn.BP(state, next_state, reward)
#                             print("  [R]  epeat this round, reward", reward, "\n   location", coordinate, "->", next_coordinate, "\n   radius", radius,"->" , next_radius)
                        if it > 1000:
                            print("--------------------------Training fail!---------------------------")
                            Goal = True
                            break
                    
                    if (Goal == True):
                        print("End searching for ", i ,", costs ", cost_it)
                        break
                if k % log_step == 0:
                    for x in rect1:
                        ax.add_patch(x)
                    ax.add_patch(rect0)
                    plt.title("loc: "+str(i)+" epoch "+str(k)+" cost total searching "+str(cost_it)+" times")
                    plt.savefig("loc_"+str(i)+"_e_"+str(k))
                    plt.close()
                    plt.cla()
                    plt.clf()
                avg_it += cost_it
            eps *= 0.95
            clear_output(wait=True)
            print("avg_it : ", avg_it/len(wifi_loc_time))
            
                        

1443
Epoch -  0 eps :  1.0
End searching for  0 , costs  30
End searching for  1 , costs  37
End searching for  2 , costs  45
End searching for  3 , costs  34
End searching for  4 , costs  71
End searching for  5 , costs  74
End searching for  6 , costs  56
End searching for  7 , costs  24
End searching for  8 , costs  69
End searching for  9 , costs  72
End searching for  10 , costs  50
End searching for  11 , costs  28
End searching for  12 , costs  29
End searching for  13 , costs  46
End searching for  14 , costs  41
End searching for  15 , costs  34
End searching for  16 , costs  56
End searching for  17 , costs  54
End searching for  18 , costs  46
End searching for  19 , costs  43
End searching for  20 , costs  53
End searching for  21 , costs  27
End searching for  22 , costs  46
End searching for  23 , costs  36
End searching for  24 , costs  74
End searching for  25 , costs  20
End searching for  26 , costs  50
End searching for  27 , costs  57
End searching for  28 , costs  

End searching for  246 , costs  67
End searching for  247 , costs  45
End searching for  248 , costs  38
End searching for  249 , costs  45
End searching for  250 , costs  31
End searching for  251 , costs  38
End searching for  252 , costs  52
End searching for  253 , costs  71
End searching for  254 , costs  78
End searching for  255 , costs  66
End searching for  256 , costs  33
End searching for  257 , costs  35
End searching for  258 , costs  43
End searching for  259 , costs  48
End searching for  260 , costs  26
End searching for  261 , costs  26
End searching for  262 , costs  50
End searching for  263 , costs  62
End searching for  264 , costs  54
End searching for  265 , costs  52
End searching for  266 , costs  125
End searching for  267 , costs  42
End searching for  268 , costs  46
End searching for  269 , costs  36
End searching for  270 , costs  70
End searching for  271 , costs  36
End searching for  272 , costs  63
End searching for  273 , costs  55
End searching for  